# Домашнее задание к лекции «Коллаборативная фильтрация»

In [28]:
from surprise import SVD, KNNBaseline, KNNWithMeans, KNNBasic, KNNWithZScore, SVDpp, SlopeOne, NMF, BaselineOnly, CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate

import pandas as pd

Загружаем данные

In [7]:
data = Dataset.load_builtin('ml-100k')

In [33]:
def fit_model(models, df, measures=['RMSE'], cv=5):        
    res = []
    
    for model in models:
        results = cross_validate(model, df, measures=measures, cv=cv, verbose=False)  

        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([str(model).split(' ')[0].split('.')[-1]], index=['Model']))
        res.append(tmp)
    
    print(pd.DataFrame(res).set_index('Model'))

Строим модели

In [36]:
fit_model([SVD(), SVDpp(verbose=False), KNNBaseline(verbose=False), KNNWithMeans(verbose=False), KNNBasic(verbose=False),
           KNNWithZScore(verbose=False), SlopeOne(), NMF(verbose=False), BaselineOnly(verbose=False),
           CoClustering(verbose=False)], data)

               test_rmse    fit_time  test_time
Model                                          
SVD             0.937092    4.764888   0.156593
SVDpp           0.919994  191.429466   3.658072
KNNBaseline     0.928943    0.918799   4.396213
KNNWithMeans    0.950730    0.542008   3.368190
KNNBasic        0.979306    0.509018   3.498884
KNNWithZScore   0.950535    0.739005   4.163612
SlopeOne        0.944763    0.832602   2.598210
NMF             0.962510    5.226774   0.177408
BaselineOnly    0.943784    0.249214   0.140399
CoClustering    0.965982    2.265007   0.135586


Как видно из таблицы, самый высокий результат получился у модели KNNBasic.

Вопросы по ДЗ:
- в ДЗ написано "получите RMSE на тестовом сете 0.87 и <b>ниже</b>". Я правильно понимаю, что это описка и имеется ввиду 0.87 и выше? Вряд ли предлагается искусственно портить результаты модели...
- Может, я что-то упустила в этом задании, но фактически это  10 строк, которые можно взять с главной страницы документации surprise, с небольшими правками на кол-во моделей и вывод результата. Так и задумано?